## 우리가 만들고자 하는 딕셔너리의 최종 모습

* Key값을 movieCd로 하는 이유는 중복 체크를 편하게 하기 위해서! 만약에 키 값이 아니면.. 또다시 반복문 ㅠ

```
{
    '20196309': {
        'movieCd': '20196309',
        'movieNm': '스파이더맨',
        'audiAcc': 6685161
    },
    '20183867': {
        'movieCd': '20183867',
        'movieNm': '알라딘',
        'audiAcc': 123213
    },
    '20196309': {
        'movieCd': '20196309',
        'movieNm': '스파이더맨',
        'audiAcc': 6685161
    }
}
```

In [ ]:
# 1. 항상 문제는 쪼개서 풀어야한다! 먼저 하루 데이터 가져와서 만들기!
import requests
key = '----'
targetDt = '20190713' #yyyymmdd
api_url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key={key}&targetDt={targetDt}'
print(api_url)
response = requests.get(api_url).json()

In [ ]:
# 2. 구조 확인하기!
response['boxOfficeResult']['weeklyBoxOfficeList']

In [ ]:
# 3. 내가 원하는 데이터 뽑아내보기!
for i in range(0, 10):
    movieCd = response['boxOfficeResult']['weeklyBoxOfficeList'][i]['movieCd']
    audiAcc = response['boxOfficeResult']['weeklyBoxOfficeList'][i]['audiAcc']
    movieNm = response['boxOfficeResult']['weeklyBoxOfficeList'][i]['movieNm']
    print(movieCd, audiAcc, movieNm)
# 뭔가 반복되는 코드가 있다? -> 나는 더 개선할 여지가 있다!
# 다시 response['boxOfficeResult']['weeklyBoxOfficeList'] 이 것을 보면 리스트!
# 리스트 안에 있는 것 다 내놔!로 표현하면 좋지 않을까. 

In [ ]:
# 4. 이제 딕셔너리를 만들어볼까?
# 만들고자하는 결과통을
movie_data = {}
# 리스트에서 꺼내주는 것은 개별 영화에 대한 데이터! 변수명을 movie로 하면 보기가 좀 더 좋다.
for movie in response['boxOfficeResult']['weeklyBoxOfficeList']:
    movie_data[movie.get('movieCd')] = {
                                        '영화코드': movie.get('movieCd'),
                                        '누적관객수': movie.get('audiAcc'),
                                        '영화이름': movie.get('movieNm')
                                    }
print(movie_data)

In [ ]:
# 5일로 늘려보자!
import datetime
import requests

for i in range(5):
    targetDt = datetime.datetime(2019, 7, 13) - datetime.timedelta(weeks=i)
    targetDt = targetDt.strftime('%Y%m%d') #yyyymmdd
    api_url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key={key}&targetDt={targetDt}&weekGb=0'
    response = requests.get(api_url).json()
    movie_data = {}
    for movie in response['boxOfficeResult']['weeklyBoxOfficeList']:
        movie_data[movie.get('movieCd')] = {
                                            '영화코드': movie.get('movieCd'),
                                            '누적관객수': movie.get('audiAcc'),
                                            '영화이름': movie.get('movieNm')
                                        }
print(movie_data)

In [ ]:
# movie_data 초기화가 되어버린다! 밖으로 꺼내자.
import datetime
import requests

key = '----'
movie_data = {}
for i in range(5):
    targetDt = datetime.datetime(2019, 7, 13) - datetime.timedelta(weeks=i)
    targetDt = targetDt.strftime('%Y%m%d') #yyyymmdd
    api_url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key={key}&targetDt={targetDt}&weekGb=0'
    response = requests.get(api_url).json()
    for movie in response['boxOfficeResult']['weeklyBoxOfficeList']:
        movie_data[movie.get('movieCd')] = {
                                            '영화코드': movie.get('movieCd'),
                                            '누적관객수': movie.get('audiAcc'),
                                            '영화이름': movie.get('movieNm')
                                        }
print(movie_data)

In [ ]:
# 값이 최신 데이터가 아니라 옛날 데이터이다. 
# 이를 해결하기 위해서는 옛날부터해서 2019년 7월 13일로 오거나
# 조건문을 추가한다!
# movie_data 초기화가 되어버린다! 밖으로 꺼내자.
import datetime
import requests

movie_data = {}
for i in range(5):
    targetDt = datetime.datetime(2019, 7, 13) - datetime.timedelta(weeks=i)
    targetDt = targetDt.strftime('%Y%m%d') #yyyymmdd
    api_url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key={key}&targetDt={targetDt}&weekGb=0'
    response = requests.get(api_url).json()
    for movie in response['boxOfficeResult']['weeklyBoxOfficeList']:
        # 무슨 값을 어디에 있는 값이랑 비교를 해야 하는지 반드시 생각해보자.
        # -> 가져온 데이터에 있는 값이 지금 누적해서 결과를 만드는 movie_data에 있는지를 확인!
        if movie.get('movieCd') not in movie_data:
            movie_data[movie.get('movieCd')] = {
                                                '영화코드': movie.get('movieCd'),
                                                '누적관객수': movie.get('audiAcc'),
                                                '영화이름': movie.get('movieNm')
                                            }
print(movie_data)

In [ ]:
# 끝났다. 이제 csv에 저장하자
import csv
with open('boxoffice.csv', 'w', encoding='utf-8') as f:
    fieldnames = ['영화코드', '누적관객수', '영화이름'] # 여기만 변경
    csv_writer = csv.DictWriter(f, fieldnames=fieldnames)
    csv_writer.writeheader()
    for item in movie_data.values(): # 딕셔너리 만든 것 반복
        csv_writer.writerow(item)

In [ ]:
# 최종 주석 없는 코드
import csv
import datetime
import requests

key = '----'
movie_data = {}
for i in range(5):
    targetDt = datetime.datetime(2019, 7, 13) - datetime.timedelta(weeks=i)
    targetDt = targetDt.strftime('%Y%m%d')
    api_url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key={key}&targetDt={targetDt}&weekGb=0'
    response = requests.get(api_url).json()
    for movie in response['boxOfficeResult']['weeklyBoxOfficeList']:
        if movie.get('movieCd') not in movie_data:
            movie_data[movie.get('movieCd')] = {
                                                '영화코드': movie.get('movieCd'),
                                                '누적관객수': movie.get('audiAcc'),
                                                '영화이름': movie.get('movieNm')
                                            }

with open('boxoffice.csv', 'w', encoding='utf-8') as f:
    fieldnames = ['영화코드', '누적관객수', '영화이름'] 
    csv_writer = csv.DictWriter(f, fieldnames=fieldnames)
    csv_writer.writeheader()
    for item in movie_data.values():
        csv_writer.writerow(item)